# Data Anaysis Project part1:

I will follow what I am learning from Google Data Analytics course to complete this project.

* <a href='http://localhost:8888/notebooks/Desktop/Chilliwack%20Real%20Estate/scrape_craigslist_for_information.ipynb#Analyze'>Ask</a>
* Prepare
* Process
* Analyze
* Share
* Act



## Overview of the project:

Analyze housing rental market in Fraser Valley to find out what is hot to real estate investors in this area. Iam going to create the code to analyze dynamically so the investors can apply in their area for their own use.  


## ASK 
    
    Find out what is 'HOT' in the market for investor, so i can find a property for them. Find out supply and demand of the rental housing market and calculate appropreate purchase price. 

#### What are the questions to ask?
Supply side:
* How much is the market rental price?
* What type of the rental is most popular and least popular?
* How many ads per day? Number of supply 
* Is rental price rising or decreasing over time?
* What is the correlation between housing price and rental price?
* From those questions, what is my prediction of the rental market? 

Demand side:
* What kind of propety does people look for rent?
* What is the price range they are looking for?
* What are the purposes of renting. (Who is looking for rent)





## Prepare  

#### Using Craigslist add to collect market data: (Supply side)
Scrape craigslist rental housing pages and store in csv file. 

<a href='https://github.com/rileypredum/East-Bay-Housing-Web-Scrape/blob/master/EB_Apt_Prices_Final.ipynb'>Code: I will use this code and modify for my situation.  </a> <br>
<a href='https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981'>Explaning</a>

## Process

I noticed that the all the values are string, so I will convert collect format as I collect the data. 

In [3]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [4]:
# Chilliwack rental housing
response = get('https://abbotsford.craigslist.org/search/apa')


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
120


In [ ]:
# Find the total number of posts to find the limit of the pagination
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
'''
each page has 119 postsso each new page is defined as follows: s=120, s=240, s=360, and so on. 
So we need to step in size 120 in the np.arange function. 
'''
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_date = []
post_city = []
num_bedroom = []
sqfts = []
prices = []
post_titles = []



In [63]:
# Show first post to look into it
post_one = posts[0]
post_one

<li class="result-row" data-pid="7564002948">
<a class="result-image gallery" data-ids="3:00D0D_56NwURftNQFz_0CI0t2,3:00x0x_fzjJHQSjxVcz_0CI0t2,3:00707_hmllmoK161mz_0CI0t2,3:01111_kzcHYMo3bWNz_0CI0t2,3:00404_hcpi8xcoAH8z_0CI0t2,3:01111_3eSAztI53CEz_0CI0t2,3:00d0d_9q1V2d3DCtwz_0CI0lM,3:00505_fxZJmZ1gpatz_0CI0lM,3:00V0V_92oigfg7KcYz_0CI0lM,3:00000_cJNzhSxcMkgz_0CI0lM,3:01717_7605xguliqiz_0CI0lM,3:00D0D_kFwMVL9UBkSz_0CI0lM,3:00X0X_EcfeRI5fG1z_0CI0lM,3:00s0s_cSGLJpkJpqbz_0lM0CI,3:00E0E_4lLcwRGiNNcz_0lM0CI,3:00c0c_8zCehLgFbTLz_0lM0CI,3:00I0I_hwObpZdSVDcz_0lM0CI,3:01111_1iBLNSsTuaLz_0lM0CI,3:00s0s_bdokfF0vEhjz_0lM0CI,3:00H0H_5kBfKsjlt3mz_0lM0CI,3:00z0z_6PRQ6JnaKK6z_0t20CI,3:00r0r_7iKRiTst1vXz_0CI0t2,3:00303_kP8vIMJNHMfz_0CI0lM,3:00k0k_6sPv4oozZajz_0CI0lM" href="https://abbotsford.craigslist.org/apa/d/abbotsford-beautiful-east-abbotsford/7564002948.html">
<span class="result-price">$3,695</span>
</a>
<div class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-rea

In [37]:
# City posted
post_city = post_one.find('span', class_='result-hood').text.strip('( )')
post_city

'Abbotsford'

In [42]:
# Number of bedroom
num_bed = int(post_one.find('span', class_='housing').text.split()[0].replace('br', ''))
num_bed, type(num_bed)

(5, int)

In [45]:
# SQFT
sqft = int(post_one.find('span', class_='housing').text.split()[2].replace('ft2', ''))
sqft, type(sqft)

(2450, int)

In [36]:
# rental price
price = int(post_one.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
price, type(price)

(3695, int)

In [38]:
# Post ad title
post_ad = post_one.find('a', class_='result-title hdrlnk').text
post_ad

'Beautiful East Abbotsford 5 bdrm House for Rent'

In [56]:
# post datetime
datetime = datetime.strptime(post_one.find('time', class_='result-date')['datetime'], '%Y-%m-%d %H:%M')
datetime, type(datetime)

(datetime.datetime(2022, 12, 5, 11, 23), datetime.datetime)

In [57]:
soup.find('div', class_='search-legend')

<div class="search-legend">
<div class="search-view js-only">
<div aria-expanded="false" aria-label="view options" class="dropdown dropdown-icons dropdown-arrows dropdown-view" role="toolbar">
<ul class="dropdown-list">
<li class="dropdown-item mode sel">
<button data-selection="list" id="listview" title="show results in a list">
<span class="view-list icon"></span>list
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="pic" id="picview" title="show results in a list with thumbnail pictures">
<span class="view-thumb icon"></span>thumb
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="grid" id="gridview" title="show results side-by-side with larger pictures">
<span class="view-gallery icon"></span>gallery
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="map" id="mapview" 

In [59]:
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
results_total

1536

In [61]:
pages = np.arange(0,results_total+1, 120)
pages

array([   0,  120,  240,  360,  480,  600,  720,  840,  960, 1080, 1200,
       1320, 1440])

## Analyze

## Share

## Act